In [7]:
import requests
import pandas as pd

# Acquisition

In [2]:
base_url = 'https://python.zach.lol'
response = request.get(base_url)
response.text

'{"api":"/api/v1","help":"/documentation"}\n'

In [3]:
api_url = base_url+ '/api/v1'
response = request.get(api_url)
api = response.json()
api

{'payload': {'routes': ['/stores',
   '/stores/{store_id}',
   '/items',
   '/items/{item_id}',
   '/sales',
   '/sales/{sale_id}']},
 'status': 'ok'}

In [4]:
api['payload']

{'routes': ['/stores',
  '/stores/{store_id}',
  '/items',
  '/items/{item_id}',
  '/sales',
  '/sales/{sale_id}']}

## Exercises

**All of the exercises for this module should be done within your ds-methodologies repository, inside of a directory named time_series.**

The end result of this exercise should be a file named acquire.py.

1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [5]:
items_url = api_url + '/items'
response = request.get(items_url)
page_1 = response.json()['payload']
page_1.keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [8]:
items = page_1['items']
items2_url = base_url + page_1['next_page']
response = requests.get(items2_url)
page_2 = response.json()['payload']
page_2.keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [9]:
items += page_2['items']
items3_url = base_url + page_2['next_page']
response = requests.get(items3_url)
page_3 = response.json()['payload']
page_3.keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [10]:
items += page_3['items']
item_df = pd.DataFrame(items)
item_df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


2. Do the same thing, but for stores.

In [11]:
items_url = api_url + '/stores'
response = request.get(items_url)
page_1 = response.json()['payload']
page_1.keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [12]:
stores = page_1['stores']
stores_df = pd.DataFrame(stores)
stores_df.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [13]:
sales_url = api_url + '/sales'
response = requests.get(sales_url)
page = response.json()['payload']
page['next_page']

'/api/v1/sales?page=2'

In [14]:
sales = []
while True:
    sales += page['sales']
    if page['next_page'] == None:
        break
    page_url = base_url + page['next_page']
    response = requests.get(page_url)
    page = response.json()['payload']
len(sales)

913000

In [15]:
sales_df = pd.DataFrame(sales)
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [16]:
item_df.to_csv('items.csv')
stores_df.to_csv('stores.csv')
sales_df.to_csv('sales.csv')

5. Combine the data from your three separate dataframes into one large dataframe.

In [17]:
sales_df.rename(columns={'store' : 'store_id', 'item':'item_id'}, inplace= True)
df = sales_df.join(stores_df.set_index('store_id'), on ='store_id')
df = df.join(item_df.set_index('item_id'), on = 'item_id')
df.head()

,item_id,sale_amount,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode,item_brand,item_name,item_price,item_upc12,item_upc14
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013


6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [18]:
opsd = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
opsd.to_csv('opsd.csv')

7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.